# Modeling

The configured `Model` in the forge will provide predefined properties of a set of Types if may contain.

## Initialisation

Run the [Blue Brain Nexus project creation notebook](https://github.com/BlueBrain/nexus-forge/blob/master/examples/notebooks/nexus-demo/00%20-%20Nexus_Project_Initialisation.ipynb) to create a Blue Brain Nexus project if you don't have one.

In [ ]:
!pip install git+https://github.com/BlueBrain/nexus-forge

In [ ]:
import getpass

The [Nexus web application](https://sandbox.bluebrainnexus.io/v1) can be used to login and get a token.

- Step 1: From the opened web page, click on the login button on the right corner and follow the instructions.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/login-ui.png)

- Step 2: At the end you’ll see a token button on the right corner. Click on it to copy the token.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/copy-token.png)


In [ ]:
token = getpass.getpass()

In [ ]:
from kgforge.core import KnowledgeGraphForge

In [ ]:
# Clone the repository if in Google Colab
import os 

!pwd
tutorial_base_dir = "./nexus-forge"
if os.path.exists(tutorial_base_dir):
  !rm -Rf $tutorial_base_dir

!git clone --single-branch https://github.com/BlueBrain/nexus-forge.git


os.chdir("/".join([tutorial_base_dir,"examples/notebooks/nexus-demo"]))

print("The working directory is now:")
!pwd

In [ ]:
# Set up some configurations

org ="tutorialnexus"
project ="myProject"
bucket = org+"/"+project
endpoint = "https://sandbox.bluebrainnexus.io/v1"


config = {
  "Model": {
    "name": "RdfModel",
    "origin": "directory",
    "source": "./neuroshapes/shapes/neurosciencegraph/commons/",
    "context": {
      "iri": "./neuroshapes_context.json"
    }
  },
  "Store": {
    "name": "BlueBrainNexus",
    "endpoint": "https://sandbox.bluebrainnexus.io/v1",
    "versioned_id_template": "{x.id}?rev={x._store_metadata._rev}",
    "file_resource_mapping": "../../configurations/nexus-store/file-to-resource-mapping.hjson"
  },
  "Formatters": {
    "identifier": "https://kg.example.ch/{}/{}"
  }
}


In [ ]:
# Get a KnowledgeGraphForge session
forge = KnowledgeGraphForge(config, endpoint=endpoint,bucket=bucket, token=token)

# Imports

In [ ]:
import json

In [ ]:
from kgforge.core import Resource

In [ ]:
from kgforge.specializations.resources import Dataset

## Prefixes

Prefixes are namespaces that are used to put Resource properties within a context.

In [ ]:
forge.prefixes()

## Types
The `type` property of a Resource can be associated to the available types in the Model. These types have a pre-defined set of properties.

In [ ]:
forge.types()

## Templates
The template will provide a set of properties for the givent type that is recomended to be used when creating Resources.

### showing the properties of a type + getting the template of a Mapping for a type

In [ ]:
forge.template("Dataset")

In [ ]:
forge.template("Dataset", only_required=True)

### creating (a) Resource instance(s)

#### manually (JSON)

In [ ]:
forge.template("Dataset", output="json", only_required=True)

In [ ]:
data = {
    "type": "Dataset",
    "name": "Awesome Analysis", 
    "description": "Awesome Dataset"
}

In [ ]:
resource_json = forge.from_json(data)

In [ ]:
print(resource_json)

#### programmatically (Dict)

In [ ]:
template = forge.template("Dataset", output="dict", only_required=True)

In [ ]:
template["name"] = "Awesome Analysis"
template["description"] =  "Awesome Dataset"

In [ ]:
resource_dict = forge.from_json(template)

In [ ]:
print(resource_dict)

## Validation
It is possible to verify that a Resource is compliant with the suggested type schema available in the Model.

### automatic status update

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
forge.validate(jane)

In [ ]:
jane._validated

In [ ]:
jane.email = "jane.doe@epfl.ch"

In [ ]:
jane._validated

### lazy actions handling

In [ ]:
! cat ../../data/persons.csv

In [ ]:
distribution = forge.attach("../../data/persons.csv")

In [ ]:
jane = Resource(
    id=forge.format("identifier", "person", "jdoe"), 
    type="Person", 
    givenName="Jane",
    familyName="Doe",
    gender="female", 
    birthDate="1985-04-20",
    distribution=distribution)

In [ ]:
print(jane)

In [ ]:
forge.validate(jane)

In [ ]:
forge.validate(jane, execute_actions_before=True)

In [ ]:
print(jane)

In [ ]:
forge.register(jane)

### error handling

Validating a Resource of type _Person_ that doesn't have an _id_ will fail since _id_ is mandatory.

In [ ]:
resource = Resource(type="Dataset", givenName="John", familyName="Smith", gender="male", birthDate="1990-04-10")

In [ ]:
forge.validate(resource)

In [ ]:
resource._last_action

In [ ]:
resource._validated